In [1]:
import cv2 as cv
import numpy as np
import os
os.chdir('../../data')

In [ ]:
cap = cv.VideoCapture('vtest.mp4')

width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

fourcc = cv.VideoWriter_fourcc('X', 'V', 'I', 'D')

outVideo = cv.VideoWriter('motionOut.avi', fourcc, 30, (width, height))

_, frame1 = cap.read()
_, frame2 = cap.read()

while cap.isOpened():
    diff = cv.absdiff(frame1, frame2)
    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)

    # remove noise
    blurFrame = cv.GaussianBlur(gray, (5, 5), 0)

    # apply threshold to get the contoures
    _, thresh = cv.threshold(blurFrame, 60, 255, cv.THRESH_BINARY)
    dilated = cv.dilate(thresh, None, iterations=10)

    contours, _ = cv.findContours(dilated, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)
    # draw = cv.drawContours(frame2, contours, -1, (0, 255, 0), 2)

    for contour in contours:
        # get the dimensions for the contour
        x, y, w, h = cv.boundingRect(contour)
        if cv.contourArea(contour) > 900:
            cv.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv.putText(frame1, 'Status: {}'.format('Movement'), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    image = cv.resize(frame1, (width, height))
    outVideo.write(image)
    cv.imshow('motion detection', image)

    frame1 = frame2
    try:
        _, frame2 = cap.read()
    except:
        break

    if cv.waitKey(60) == 27:
        break

cv.destroyAllWindows()
cap.release()
outVideo.release()